**IFSP - Campus Campinas** <br>
**Pós-graduação em Ciência de Dados** <br>

**TCC - Avaliação de Projetos de Lei usando técnicas de NLP/ML**

#### Sprint 4 - v 1.4 
- continuando com dados salvos no Sprint 3, já pre-processados com SpaCy
- pre-processamento com Neattext na coluna **'Ementa'** da PLO
- feature extraction com TfidfVectorizer na coluna **'Ementa'** da PLO
- treinamento com LogisticRegression
- rodado em PC local (não AWS)

Ref: Natural Language Processing (NLP) 2023.1 - IFSP Campinas <br>
https://github.com/prof-samuka-IFSP/IFSP-CMP-D3TOP-2023.1/tree/main

Orientadora: Bianca Maria Pedrosa - IFSP Campinas<br>
Aluno: Swift Motoo Yaguchi - CP301665X

------

### 1. Etapa de limpeza e pré-processamento

##### Instalando bibliotecas

In [1]:
#Upgrade dependencies
#!pip install --upgrade pip
#!pip install --upgrade scikit-learn
!pip install --upgrade wordcloud
#!pip install --upgrade sagemaker

In [2]:
import pandas as pd
import numpy as np

##### Leitura da base de dados salvo no Sprint 3

In [3]:
# load the datasets
df_train = pd.read_csv('test_dataset_train_3.csv', sep=';', index_col=False)
df_test = pd.read_csv('test_dataset_test_3.csv', sep=';', index_col=False)

In [4]:
df_train

,Vereador,Partido,Aprovado,Ementa,Texto,Label
0,Carlão do PT,PT,sim,denomina centro saúde acylino souza santos cen...,art 1 fica denominado centro saúde acylino sou...,1
1,Marcos Bernardelli,PSDB,sim,denomina avenida dr roberto cerqueira oliveira...,art 1 0 fica denominada avenida dr roberto cer...,1
2,Luiz Cirilo,PSDB,sim,proíbe estabelecimentos comerciais situados mu...,art 1 estabelecimentos comerciais situados mun...,1
3,Luiz Cirilo,PSDB,sim,institui programa conscientização prevenção al...,art 1 2 fica instituído município programa con...,1
4,Carmo Luiz,PSC,sim,denomina praça cyro baldin recreio município c...,art 1º fica denominado praça cyro baldin recre...,1
...,...,...,...,...,...,...
254,Juscelino da Barbarense,PP,não,obriga postos gasolina município campinas impl...,art i p postos gasolina município campinas fic...,0
255,Cidão Santos,PROS,não,dispõe implantação programa educação comportam...,art 1 2 secretaria municipal educação será res...,0
256,Nelson Hossri,PTN,não,dispõe criação bolsões eou espaços públicos ad...,09102019 conforme manifestação procuradoria le...,0
257,Jorge da Farmácia,PSDB,sim,reconhece âmbito município campinas visão mono...,art 10 fica reconhecida deficiência visual â...,1


In [5]:
df_test

,Vereador,Partido,Aprovado,Ementa,Texto,Label
0,Jorge da Farmácia,PSDB,sim,declara órgão utilidade pública municipal fund...,art i p fica declarada órgão utilidade pública...,1
1,Carmo Luiz,PSC,não,institui homenagem mulheres vítimas feminicídi...,art i p fica instituída homenagem mulheres vít...,0
2,Zé Carlos,PSB,sim,denomina rua osvualte darri via pública municí...,art 1 fica denominada rua osvualte darri rua 0...,1
3,Luiz Rossini,PV,sim,inclui calendário oficial município campinas s...,art 1 2 fica incluída calendário oficial munic...,1
4,Cláudio da Farmácia,PSC,não,dispõe criação bolsões estacionamento exclusiv...,16092019 conforme manifestação procuradoria le...,0
...,...,...,...,...,...,...
106,Pr. Elias Azevedo,PSB,não,dispõe criação bolsões estacionamento exclusiv...,16092019 conforme manifestação procuradoria le...,0
107,Tenente Santini,PSD,não,estabelece critérios serem observados nomeação...,art i p nomeação cargos presidente diretor adm...,0
108,Marcos Bernardelli,PSDB,não,dispõe âmbito município campinas obrigatorieda...,art 10 ficam agências bancárias estabelecidas ...,0
109,Professor Alberto,PR,sim,altera art 1º lei nº 14045 07 abril 2011 inclu...,art 1º fica alterado art 1º lei nº 14045 7 abr...,1


In [6]:
print('O tamanho do dataset de treino é:', df_train.shape)
print('O tamanho do dataset de teste é:', df_test.shape)

O tamanho do dataset de treino é: (259, 6)
O tamanho do dataset de teste é: (111, 6)


##### Nova remoção na coluna 'Texto' usando biblioteca neattext
- lowering, expand contractions
- remove:
  + punctuations, stop words, urls, emails, numbers, emojis, phone numbers, multiple whitespaces, currency symbols, special characters

In [7]:
!pip install neattext

In [8]:
import neattext.functions as ntx

def text_preprocessing(text_in: str) -> str:
    text = text_in.lower()
    text = ntx.fix_contractions(text)
    text = ntx.remove_punctuations(text)
    text = ntx.remove_stopwords(text)
    text = ntx.remove_urls(text)
    text = ntx.remove_emails(text)
    text = ntx.remove_numbers(text)
    text = ntx.remove_emojis(text)
    text = ntx.remove_phone_numbers(text)
    text = ntx.remove_multiple_spaces(text)
    text = ntx.remove_currency_symbols(text)
    text = ntx.remove_special_characters(text)
    return text

In [9]:
# progress bar in pandas
!pip install tqdm

In [10]:
from tqdm import tqdm
tqdm.pandas()  # it enables some new progress bar functions/methods for pandas

In [11]:
# pre-process the training set
df_train['Ementa-pre'] = df_train['Ementa'].progress_apply(lambda text: text_preprocessing(text))

100%|███████████████████████████████████████████████████████████████████████████████| 259/259 [00:00<00:00, 515.23it/s]


In [12]:
df_train.head()

,Vereador,Partido,Aprovado,Ementa,Texto,Label,Ementa-pre
0,Carlão do PT,PT,sim,denomina centro saúde acylino souza santos cen...,art 1 fica denominado centro saúde acylino sou...,1,denomina centro sade acylino souza santos cent...
1,Marcos Bernardelli,PSDB,sim,denomina avenida dr roberto cerqueira oliveira...,art 1 0 fica denominada avenida dr roberto cer...,1,denomina avenida dr roberto cerqueira oliveira...
2,Luiz Cirilo,PSDB,sim,proíbe estabelecimentos comerciais situados mu...,art 1 estabelecimentos comerciais situados mun...,1,probe estabelecimentos comerciais situados mun...
3,Luiz Cirilo,PSDB,sim,institui programa conscientização prevenção al...,art 1 2 fica instituído município programa con...,1,institui programa conscientizao preveno aliena...
4,Carmo Luiz,PSC,sim,denomina praça cyro baldin recreio município c...,art 1º fica denominado praça cyro baldin recre...,1,denomina praa cyro baldin recreio municpio cam...


In [13]:
# pre-process the training set
df_test['Ementa-pre'] = df_test['Ementa'].progress_apply(lambda text: text_preprocessing(text))

100%|███████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 925.56it/s]


In [14]:
df_test.head()

,Vereador,Partido,Aprovado,Ementa,Texto,Label,Ementa-pre
0,Jorge da Farmácia,PSDB,sim,declara órgão utilidade pública municipal fund...,art i p fica declarada órgão utilidade pública...,1,declara rgo utilidade pblica municipal fundao ...
1,Carmo Luiz,PSC,não,institui homenagem mulheres vítimas feminicídi...,art i p fica instituída homenagem mulheres vít...,0,institui homenagem mulheres vtimas feminicdio ...
2,Zé Carlos,PSB,sim,denomina rua osvualte darri via pública municí...,art 1 fica denominada rua osvualte darri rua 0...,1,denomina rua osvualte darri pblica municpio ca...
3,Luiz Rossini,PV,sim,inclui calendário oficial município campinas s...,art 1 2 fica incluída calendário oficial munic...,1,inclui calendrio oficial municpio campinas sem...
4,Cláudio da Farmácia,PSC,não,dispõe criação bolsões estacionamento exclusiv...,16092019 conforme manifestação procuradoria le...,0,dispe criao bolses estacionamento exclusivos m...


### 3. Feature Extraction

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(df_train['Ementa-pre'])
y_train = df_train['Label']

X_test = tfidf.transform(df_test['Ementa-pre'])
y_test = df_test['Label']

In [16]:
X_train.shape, X_test.shape

((259, 1199), (111, 1199))

In [17]:
print(f'Vocabulary size: {len(tfidf.vocabulary_)}')

Vocabulary size: 1199


### 4. Treinamento de modelos

In [18]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(class_weight='balanced', n_jobs=-1)

logreg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', n_jobs=-1)

In [19]:
# prediction on training set
y_train_pred = logreg.predict(X_train)

In [20]:
from sklearn.metrics import classification_report

#print(classification_report(y_train, y_train_pred, target_names=target_names))
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       132
           1       0.97      0.92      0.95       127

    accuracy                           0.95       259
   macro avg       0.95      0.95      0.95       259
weighted avg       0.95      0.95      0.95       259



In [21]:
from sklearn.metrics import f1_score

f1_train = f1_score(y_train, y_train_pred, average='macro')from sklearn.metrics import balanced_accuracy_score

balacc_train = balanced_accuracy_score(y_train, y_train_pred)

print(f'Balanced Acc Train: {balacc_train}')

print(f'F1 Train: {f1_train}')

F1 Train: 0.9496989706739172


In [22]:
from sklearn.metrics import balanced_accuracy_score

balacc_train = balanced_accuracy_score(y_train, y_train_pred)

print(f'Balanced Acc Train: {balacc_train}')

Balanced Acc Train: 0.9492662848962061


#### Evaluate the model on the Test Set

In [23]:
# prediction on testing set
y_test_pred = logreg.predict(X_test)

In [24]:
#from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85        57
           1       0.82      0.91      0.86        54

    accuracy                           0.86       111
   macro avg       0.86      0.86      0.86       111
weighted avg       0.86      0.86      0.86       111



In [25]:
from sklearn.metrics import f1_score
f1_test = f1_score(y_test, y_test_pred, average='macro')
print(f'F1 Test: {f1_test}')

F1 Test: 0.8557504873294347


O resultado **F1 score** diminuiu um pouco em relação ao Sprint 3, devido ao menor vocabulário no campo 'Ementa' em relação ao campo "Texto'

In [26]:
from sklearn.metrics import balanced_accuracy_score

balacc_test = balanced_accuracy_score(y_test, y_test_pred)

print(f'Balanced Acc Train: {balacc_test}')

Balanced Acc Train: 0.8572124756335282


Os resultados dos testes de classificação no sprint 4 são ligeiramente piores do que os resultados dos testes de classificação no sprint 3. A precisão, o recall e o F1-score do modelo no conjunto de teste diminuíram em 1%, 4% e 3%, respectivamente. A acurácia do modelo no conjunto de teste também diminuiu em 3%.

Essas quedas nos resultados podem ser devido a um dos seguintes fatores:

- Inclusão da ementa: A ementa pode não ser uma característica tão importante quanto o texto do projeto de lei para a classificação de projetos de lei.<br>
- Tamanho do conjunto de teste: O tamanho do conjunto de teste é relativamente pequeno (111 exemplos). Isso significa que os resultados podem ser mais propensos a flutuações aleatórias.